In [1]:
import os
import duckdb
import fiona
import pandas as pd
from typing import List, Optional

In [2]:
db = r'W:\lwbc\visr\Workarea\moez_labiadh\LEARNING\duckdb\hd_test.db'
conn = duckdb.connect(db)
conn.install_extension('spatial')
conn.load_extension('spatial')

In [13]:
def import_fgdb_layers(
    conn: duckdb.DuckDBPyConnection,
    fgdb_path: str,
    feature_classes: Optional[List[str]] = None,
) -> pd.DataFrame:
    """
    Import layers from an ESRI File Geodatabase into DuckDB as tables.
    The geometry column name will be standardized to 'geometry',
    
    Parameters
    ----------
    conn : duckdb.DuckDBPyConnection
        An active DuckDB connection.
    gdb_path : str
        Filesystem path to the .gdb directory.
    feature_classes : list of str, optional
        Names of feature classes to import. If None, all layers in the FGDB
        will be imported.

    """
    # If no specific list provided, list all layers in the FGDB
    if feature_classes is None:
        feature_classes = fiona.listlayers(fgdb_path)

    total = len(feature_classes)

    # Grab existing tables in DuckDB
    existing_tables = {
        row[0] for row in conn.execute("SHOW TABLES").fetchall()
    }

    stats_list = []
    for idx, layer in enumerate(feature_classes, start=1):
        print(f"\nProcessing layer {idx} of {total}: '{layer}'.")

        # Delete table if it already exists
        if layer in existing_tables:
            print("...layer exists in DB — deleting it...")
            conn.execute(f'DROP TABLE IF EXISTS "{layer}";')

        # Import the layer into DuckDB
        print("...importing into the DB...")
        conn.execute(f"""
            CREATE TABLE "{layer}" AS
            SELECT *
            FROM ST_Read(
                '{fgdb_path}',
                allowed_drivers => ['OpenFileGDB'],
                layer           => '{layer}'
            );
        """)

        # Rename the geometry column to 'geometry'
        cols    = conn.execute(f"PRAGMA table_info('{layer}')").fetchall()
        geomcol = next(col[1] for col in cols if col[2].upper() == 'GEOMETRY')

        conn.execute(
            f'ALTER TABLE "{layer}" RENAME COLUMN {geomcol} TO geometry;'
        )

        # Build spatial index
        print("...creating RTREE index on geometry…")
        conn.execute(
            f"""
            DROP INDEX IF EXISTS idx_geo_{layer};
            CREATE INDEX idx_geo_{layer} ON {layer} USING RTREE (geometry);
            """
        )

        # collect stats
        row_count = conn.execute(f'SELECT COUNT(*) FROM "{layer}";').fetchone()[0]
        col_count = len(cols)
        stats_list.append({
            'table_name': layer,
            'row_count': row_count,
            'column_count': col_count,
            'geometry_column': 'geometry'
        })

    return pd.DataFrame(stats_list)

In [14]:
fgdb_path = r'W:\ilmb\dss\projects\GeoBC\Human Disturbance\outputs\2025\Disturbance_1_Raw.gdb'
layer_list = ['Power_2_Flooded_Reservoirs_raw', 'Power_3_Transmission_Lines_GBA_raw']

import_log = import_fgdb_layers (conn, fgdb_path, layer_list)
import_log


Processing layer 1 of 2: 'Power_2_Flooded_Reservoirs_raw'.
...layer exists in DB — deleting it...
...importing into the DB...
...creating RTREE index on geometry…

Processing layer 2 of 2: 'Power_3_Transmission_Lines_GBA_raw'.
...layer exists in DB — deleting it...
...importing into the DB...
...creating RTREE index on geometry…


,table_name,row_count,column_count,geometry_column
0,Power_2_Flooded_Reservoirs_raw,41,31,geometry
1,Power_3_Transmission_Lines_GBA_raw,2002,22,geometry


In [16]:
fgdb_path = r'W:\ilmb\dss\projects\GeoBC\Human Disturbance\data\00_Tile_Grids.gdb'
layer_list = ['tiles_250k_with_94A_custom']

import_log = import_fgdb_layers (conn, fgdb_path, layer_list)
import_log


Processing layer 1 of 1: 'tiles_250k_with_94A_custom'.
...layer exists in DB — deleting it...
...importing into the DB...
...creating RTREE index on geometry…


,table_name,row_count,column_count,geometry_column
0,tiles_250k_with_94A_custom,92,9,geometry


In [38]:
shapefile_path = r'W:\lwbc\visr\Workarea\moez_labiadh\TOOLS\SCRIPTS\STATUSING\test_data\aoi_test_7_vvvbig.shp'
table_name = os.path.basename(shapefile_path).replace('.shp', '')

conn.execute(f"""
 DROP TABLE IF EXISTS "{table_name}";
CREATE TABLE "{table_name}" AS
SELECT *
FROM ST_Read(
    '{shapefile_path}',
    allowed_drivers => ['ESRI Shapefile']
);
"""
)

In [17]:
tabs= conn.execute("""SHOW TABLES""").df()
tabs

,name
0,Power_2_Flooded_Reservoirs_raw
1,Power_3_Transmission_Lines_GBA_raw
2,aoi_test_7_vvvbig
3,tiles_250k_with_94A_custom
